In [1]:
import pandas as pd

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics
import pickle

In [3]:
print("Tensorlfow version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Tensorlfow version:  2.12.0
Eager mode:  True
GPU is available


In [3]:
# reading the CSV file
data = pd.read_csv('VDISC_train1.csv')
# displaying the contents of the CSV file


C:\Users\unik0\AppData\Local\Temp\ipykernel_13328\542229800.py:2: DtypeWarning: Columns (0,1,2,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('VDISC_train1.csv')


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data.head()

,CWE-119,CWE-120,CWE-469,CWE-476,CWE-other,functionSource,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48
0,True,True,False,False,False,b'__GEOIP_PREPARE_TEREDO(geoipv6_t * v6)\n{\n ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,True,False,False,False,b'draw_keys (int draw)\n{\n char s...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,True,True,False,False,b'save_weblist()\n{\n\tchar filename[512];\n\n...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,True,False,False,False,"b'memory_ass_sub(PyMemoryViewObject *self, PyO...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,True,False,False,False,b'write_metadata_block_data_streaminfo_cb_(FLA...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
columns = data.columns
print(columns)
#data= data.replace({'CWE-119':{'FALSE': False, 'TRUE':True}})

Index(['CWE-119', ' CWE-120', 'CWE-469', 'CWE-476', 'CWE-other',
       'functionSource', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40',
       'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44',
       'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48'],
      dtype='object')


In [6]:
data_cropped = data.loc[:,['CWE-119', ' CWE-120', 'CWE-469', 'CWE-476', 'CWE-other','functionSource']]

In [8]:
#data_cropped.head(10)
columns = list(data_cropped.columns)
data_cropped = data_cropped.replace({True:"TRUE",False:"FALSE"})
for column in columns[:-1]:
  data_cropped[column] = data_cropped[column].map(str.upper)

In [9]:
data_cropped=data_cropped.sample(frac=1)

In [10]:
data_cropped.head()

,CWE-119,CWE-120,CWE-469,CWE-476,CWE-other,functionSource
50873,FALSE,TRUE,FALSE,FALSE,FALSE,b'qlcnic_sysfs_get_port_stats(struct file *fil...
35210,FALSE,TRUE,FALSE,FALSE,FALSE,b'drm_dp_sideband_parse_remote_i2c_read_ack(st...
45622,TRUE,TRUE,FALSE,FALSE,TRUE,"b'create_inet_socket()\n{\n\tint fd, on = 1;\n..."
45676,FALSE,TRUE,FALSE,FALSE,FALSE,"b'show_in(struct device *dev, struct device_at..."
12437,FALSE,TRUE,FALSE,FALSE,FALSE,"b""expand_variables(char **buff, const char *sr..."


In [11]:
myrand=71926
np.random.seed(myrand)
tf.random.set_seed(myrand)
print("Random seed is:",myrand)

Random seed is: 71926


In [12]:
### Set the global value
WORDS_SIZE=10000
INPUT_SIZE=500
NUM_CLASSES=2
MODEL_NUM=0
EPOCHS=10

In [13]:
data_cropped.to_pickle("VDISC_train1.pickle")

In [14]:
data=pd.read_pickle("VDISC_train1.pickle")

In [15]:
x_all = data_cropped['functionSource']

In [16]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=False)
tokenizer.fit_on_texts(list(x_all))
tokenizer.num_words = WORDS_SIZE

In [17]:
# data['functionSource'] =  [x.decode('utf-8') for x in data['functionSource'] ]
list_tokenized_train = tokenizer.texts_to_sequences(data['functionSource'])
x_train = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, 
                                  maxlen=INPUT_SIZE,
                                  padding='post')
x_train = x_train.astype(np.int64)

In [50]:
import pickle

#saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
#loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer= pickle.load(handle)

In [18]:
y_train=[]

In [19]:
columns = list(data.columns[:-1])
for column in columns:
        data[column] = data[column].map({"FALSE": 0, "TRUE": 1})

In [20]:
data.head(10)

,CWE-119,CWE-120,CWE-469,CWE-476,CWE-other,functionSource
50873,0.0,1.0,0.0,0.0,0.0,b'qlcnic_sysfs_get_port_stats(struct file *fil...
35210,0.0,1.0,0.0,0.0,0.0,b'drm_dp_sideband_parse_remote_i2c_read_ack(st...
45622,1.0,1.0,0.0,0.0,1.0,"b'create_inet_socket()\n{\n\tint fd, on = 1;\n..."
45676,0.0,1.0,0.0,0.0,0.0,"b'show_in(struct device *dev, struct device_at..."
12437,0.0,1.0,0.0,0.0,0.0,"b""expand_variables(char **buff, const char *sr..."
34481,0.0,0.0,0.0,1.0,0.0,"b'tsubst_aggr_type (tree t,\n\t\t tree args,\..."
6444,0.0,1.0,0.0,0.0,0.0,b'crypto_rfc4106_setkey(struct crypto_aead *pa...
335,0.0,0.0,0.0,0.0,1.0,"b'unix_wait(value unit)\n{\n int pid, status;..."
14049,1.0,1.0,0.0,0.0,0.0,"b'fetch_chunked(struct worker *wrk, struct htt..."
1515,0.0,0.0,0.0,0.0,1.0,"b'eet_pbkdf2_sha1(const char *key,\n ..."


In [21]:
#  Create a random weights matrix
random_weights = np.random.normal(size=(WORDS_SIZE, 13),scale=0.01)

In [22]:
inp_layer = tf.keras.layers.Input(shape=(INPUT_SIZE,))
mid_layers = tf.keras.layers.Embedding(input_dim = WORDS_SIZE,
                                    output_dim = 13,
                                    weights=[random_weights],
                                    input_length = INPUT_SIZE)(inp_layer)
mid_layers = tf.keras.layers.Convolution1D(filters=512, kernel_size=(9), padding='same', activation='relu')(mid_layers)
mid_layers = tf.keras.layers.MaxPool1D(pool_size=5)(mid_layers)
mid_layers = tf.keras.layers.Dropout(0.5)(mid_layers)
mid_layers = tf.keras.layers.Flatten()(mid_layers)
mid_layers = tf.keras.layers.Dense(64, activation='relu')(mid_layers)
mid_layers = tf.keras.layers.Dense(16, activation='relu')(mid_layers)
output1 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output2 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output3 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output4 =tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output5 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
model = tf.keras.Model(inp_layer,[output1,output2,output3,output4,output5])

# Define custom optimizers
# adam = tf.keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1, decay=0.0, amsgrad=False)

## Compile model with metrics
model.compile(optimizer="ADAM", loss='categorical_crossentropy', metrics=['accuracy'])
print("CNN model built: ")
model.summary()

CNN model built: 
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 500, 13)      130000      ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 500, 512)     60416       ['embedding[0][0]']              
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 100, 512)     0           ['conv1d[0][0]']                 
                                                                            

In [23]:
## Create TensorBoard callbacks

callbackdir= 'D:\\temp\\cb'

tbCallback = tf.keras.callbacks.TensorBoard(log_dir=callbackdir, 
                         histogram_freq=1,
                         embeddings_freq=1,
                         write_graph=True, 
                         write_images=True)

tbCallback.set_model(model)
mld = 'model/model-ALL-{epoch:02d}.hdf5'

## Create best model callback
mcp = tf.keras.callbacks.ModelCheckpoint(filepath=mld, 
                                         monitor="val_loss",
                                         save_best_only=True, 
                                         mode='auto', 
                                         save_freq='epoch', 
                                         verbose=1)

In [24]:
# print(data.columns)
y_train= np.array([data['CWE-119'].values,data[' CWE-120'].values,data['CWE-469'].values,data['CWE-476'].values,data['CWE-other'].values])
y_train

array([[0., 0., 1., ..., 0., 1., 0.],
       [1., 1., 1., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 1., 1.]])

In [25]:
y_train=[]
#print(data.iloc[:,2])
data = data.ffill()
columns = data.columns[:-1]
for column in columns:
    y_train.append(tf.keras.utils.to_categorical(list(data[column].values), num_classes=NUM_CLASSES).astype(np.int64))
y_train = np.array(y_train)

In [26]:
y_train[4]

array([[1, 0],
       [1, 0],
       [0, 1],
       ...,
       [1, 0],
       [0, 1],
       [0, 1]], dtype=int64)

In [28]:
class_weights = [{0: 1., 1: 5.},{0: 1., 1: 5.},{0: 1., 1: 5.},{0: 1., 1: 5.},{0: 1., 1: 5.}]

callback= tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=10, verbose=1, )
history = model.fit(x = x_train,
          y = [y_train[0],y_train[1],y_train[2],y_train[3],y_train[4]],
          epochs = 40,
          batch_size = 128,
          verbose =2,
          callbacks=[callback])

with open('history/History-ALL-40EP-CNN', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

Epoch 1/40
517/517 - 104s - loss: 1.2705 - dense_2_loss: 0.2689 - dense_3_loss: 0.3344 - dense_4_loss: 0.1136 - dense_5_loss: 0.1806 - dense_6_loss: 0.3731 - dense_2_accuracy: 0.8927 - dense_3_accuracy: 0.8612 - dense_4_accuracy: 0.9684 - dense_5_accuracy: 0.9331 - dense_6_accuracy: 0.8363 - 104s/epoch - 202ms/step
Epoch 2/40
517/517 - 105s - loss: 1.1084 - dense_2_loss: 0.2243 - dense_3_loss: 0.2903 - dense_4_loss: 0.1008 - dense_5_loss: 0.1598 - dense_6_loss: 0.3333 - dense_2_accuracy: 0.9126 - dense_3_accuracy: 0.8800 - dense_4_accuracy: 0.9691 - dense_5_accuracy: 0.9406 - dense_6_accuracy: 0.8572 - 105s/epoch - 203ms/step
Epoch 3/40
517/517 - 105s - loss: 0.9799 - dense_2_loss: 0.1912 - dense_3_loss: 0.2545 - dense_4_loss: 0.0903 - dense_5_loss: 0.1420 - dense_6_loss: 0.3020 - dense_2_accuracy: 0.9249 - dense_3_accuracy: 0.8951 - dense_4_accuracy: 0.9704 - dense_5_accuracy: 0.9483 - dense_6_accuracy: 0.8713 - 105s/epoch - 203ms/step
Epoch 4/40
517/517 - 105s - loss: 0.8688 - dense_

Epoch 15/40
517/517 - 102s - loss: 0.3287 - dense_2_loss: 0.0586 - dense_3_loss: 0.0832 - dense_4_loss: 0.0341 - dense_5_loss: 0.0412 - dense_6_loss: 0.1116 - dense_2_accuracy: 0.9783 - dense_3_accuracy: 0.9681 - dense_4_accuracy: 0.9879 - dense_5_accuracy: 0.9844 - dense_6_accuracy: 0.9569 - 102s/epoch - 197ms/step
Epoch 16/40
517/517 - 102s - loss: 0.3077 - dense_2_loss: 0.0540 - dense_3_loss: 0.0766 - dense_4_loss: 0.0326 - dense_5_loss: 0.0391 - dense_6_loss: 0.1055 - dense_2_accuracy: 0.9797 - dense_3_accuracy: 0.9710 - dense_4_accuracy: 0.9886 - dense_5_accuracy: 0.9857 - dense_6_accuracy: 0.9593 - 102s/epoch - 197ms/step
Epoch 17/40
517/517 - 101s - loss: 0.2971 - dense_2_loss: 0.0523 - dense_3_loss: 0.0750 - dense_4_loss: 0.0300 - dense_5_loss: 0.0370 - dense_6_loss: 0.1029 - dense_2_accuracy: 0.9816 - dense_3_accuracy: 0.9713 - dense_4_accuracy: 0.9896 - dense_5_accuracy: 0.9868 - dense_6_accuracy: 0.9607 - 101s/epoch - 196ms/step
Epoch 18/40
517/517 - 101s - loss: 0.2819 - de

Epoch 29/40
517/517 - 101s - loss: 0.1813 - dense_2_loss: 0.0341 - dense_3_loss: 0.0459 - dense_4_loss: 0.0169 - dense_5_loss: 0.0248 - dense_6_loss: 0.0597 - dense_2_accuracy: 0.9878 - dense_3_accuracy: 0.9834 - dense_4_accuracy: 0.9940 - dense_5_accuracy: 0.9916 - dense_6_accuracy: 0.9776 - 101s/epoch - 195ms/step
Epoch 30/40
517/517 - 101s - loss: 0.1804 - dense_2_loss: 0.0343 - dense_3_loss: 0.0466 - dense_4_loss: 0.0165 - dense_5_loss: 0.0237 - dense_6_loss: 0.0593 - dense_2_accuracy: 0.9876 - dense_3_accuracy: 0.9833 - dense_4_accuracy: 0.9940 - dense_5_accuracy: 0.9918 - dense_6_accuracy: 0.9782 - 101s/epoch - 196ms/step
Epoch 31/40
517/517 - 101s - loss: 0.1733 - dense_2_loss: 0.0325 - dense_3_loss: 0.0445 - dense_4_loss: 0.0156 - dense_5_loss: 0.0222 - dense_6_loss: 0.0585 - dense_2_accuracy: 0.9884 - dense_3_accuracy: 0.9837 - dense_4_accuracy: 0.9945 - dense_5_accuracy: 0.9918 - dense_6_accuracy: 0.9789 - 101s/epoch - 195ms/step
Epoch 32/40
517/517 - 102s - loss: 0.1755 - de

FileNotFoundError: [Errno 2] No such file or directory: 'history/History-ALL-40EP-CNN'

In [33]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
# reading the CSV file
data_test = pd.read_csv('VDISC_test1.csv')
# displaying the contents of the CSV file


In [30]:
data_cropped = data_test.loc[:,['CWE-119', 'CWE-120', 'CWE-469', 'CWE-476', 'CWE-other','functionSource']]

In [34]:
data_cropped

,CWE-119,CWE-120,CWE-469,CWE-476,CWE-other,functionSource
991,FALSE,FALSE,FALSE,FALSE,TRUE,b'camp_verify_tree()\n{\n int i;\n Camp_...
6881,TRUE,TRUE,FALSE,FALSE,FALSE,b'reg_vpi_release(register struct net_t *np)\n...
7875,FALSE,TRUE,FALSE,FALSE,FALSE,"b'snd_component_add(struct snd_card *card, con..."
123,FALSE,FALSE,FALSE,FALSE,TRUE,"b'xdr_exports(XDR *xdrs, exports *objp)\n{\n\n..."
3822,TRUE,TRUE,FALSE,FALSE,TRUE,"b'PQprintTuples(const PGresult *res,\n\t\t\t ..."
...,...,...,...,...,...,...
2601,FALSE,FALSE,FALSE,TRUE,FALSE,"b'cached_nonzero_bits (const_rtx x, enum machi..."
7705,TRUE,TRUE,FALSE,FALSE,FALSE,"b'AT_SetSMSMemoryType(gn_memory_type mt, struc..."
0,FALSE,FALSE,FALSE,TRUE,FALSE,b'buildToken( char *text )\n#else\nbuildToken(...
1218,TRUE,TRUE,FALSE,FALSE,FALSE,b'shell_version_string ()\n{\n static char tt...


In [32]:
#data_cropped.head(10)
columns = list(data_cropped.columns)
data_cropped = data_cropped.replace({True:"TRUE",False:"FALSE"})
for column in columns[:-1]:
  data_cropped[column] = data_cropped[column].map(str.upper)

In [33]:
data_cropped=data_cropped.sample(frac=1)

In [35]:
data_cropped.to_pickle("VDISC_test1.pickle")

In [36]:
data_test= pd.read_pickle("VDISC_test1.pickle")

In [37]:
# data['functionSource'] =  [x.decode('utf-8') for x in data['functionSource'] ]
list_tokenized_test = tokenizer.texts_to_sequences(data_test['functionSource'])
x_test = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_test, 
                                  maxlen=INPUT_SIZE,
                                  padding='post')
x_test = x_test.astype(np.int64)

In [38]:
x_test.shape

(8322, 500)

In [39]:
columnstest = list(data_test.columns[:-1])
for column in columnstest:
        data_test[column] = data_test[column].map({"FALSE": 0, "TRUE": 1})

In [40]:
data_test

,CWE-119,CWE-120,CWE-469,CWE-476,CWE-other,functionSource
991,0,0,0,0,1,b'camp_verify_tree()\n{\n int i;\n Camp_...
6881,1,1,0,0,0,b'reg_vpi_release(register struct net_t *np)\n...
7875,0,1,0,0,0,"b'snd_component_add(struct snd_card *card, con..."
123,0,0,0,0,1,"b'xdr_exports(XDR *xdrs, exports *objp)\n{\n\n..."
3822,1,1,0,0,1,"b'PQprintTuples(const PGresult *res,\n\t\t\t ..."
...,...,...,...,...,...,...
2601,0,0,0,1,0,"b'cached_nonzero_bits (const_rtx x, enum machi..."
7705,1,1,0,0,0,"b'AT_SetSMSMemoryType(gn_memory_type mt, struc..."
0,0,0,0,1,0,b'buildToken( char *text )\n#else\nbuildToken(...
1218,1,1,0,0,0,b'shell_version_string ()\n{\n static char tt...


In [41]:
# print(data.columns)
y_test= np.array([data['CWE-119'].values,data[' CWE-120'].values,data['CWE-469'].values,data['CWE-476'].values,data['CWE-other'].values])
y_test.shape

(5, 66120)

In [42]:
y_test=[]
#print(data.iloc[:,2])
data_test = data_test.ffill()
columns = data_test.columns[:-1]
for column in columns:
    y_test.append(tf.keras.utils.to_categorical(list(data_test[column].values), num_classes=NUM_CLASSES).astype(np.int64))
y_test = np.array(y_test)

In [43]:
y_test

array([[[1, 0],
        [0, 1],
        [1, 0],
        ...,
        [1, 0],
        [0, 1],
        [1, 0]],

       [[1, 0],
        [0, 1],
        [0, 1],
        ...,
        [1, 0],
        [0, 1],
        [1, 0]],

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [0, 1]],

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [0, 1],
        [1, 0],
        [1, 0]],

       [[0, 1],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [0, 1]]], dtype=int64)

In [ ]:
# Load model
model = tf.keras.models.load_model("svd_1.hdf5")

In [45]:
results = model.evaluate(x_test, y_test = [y_test[0],y_test[1],y_test[2],y_test[3],y_test[4]], batch_size=128)
for num in range(0,len(model.metrics_names)):
    print(model.metrics_names[num]+': '+str(results[num]))

TypeError: Invalid keyword arguments: ['y_test']

In [46]:
predicted = model.predict(x_test)

261/261 [==============================] - 5s 13ms/step


In [68]:
from sklearn.metrics import confusion_matrix

In [73]:
y_test[0]

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [0, 1],
       [1, 0]])

In [76]:
data_test

,CWE-119,CWE-120,CWE-469,CWE-476,CWE-other,functionSource
6185,1,1,0,0,0,"b'_compare_ints(_cs_io_t *inp1,\n ..."
5506,0,0,0,0,1,"b'fits_write_header(const qfits_header* hdr, c..."
1201,0,1,0,0,0,b'server_unlink (rpcsvc_request_t *req)\n{\n ...
3064,1,1,0,0,0,"b""hexdump(unsigned char *s, unsigned long l)\n..."
1077,0,0,0,0,1,b'GLESendSocket(const string& commands) {\n#if...
...,...,...,...,...,...,...
1727,0,0,0,1,0,b'qpol_constraint_get_perm_iter(const qpol_pol...
5756,0,1,0,0,1,"b'main(int argc, char ** argv)\n{\n dxml_elem..."
5697,0,1,0,0,0,"b""cob_move_alphanum_to_alphanum (cob_field *f1..."
8159,1,1,0,0,0,"b'git_path_find_dir(git_buf *dir, const char *..."


In [47]:
model.evaluate(x_test,y = [y_test[0],y_test[1],y_test[2],y_test[3],y_test[4]])

261/261 [==============================] - 4s 13ms/step - loss: 4.6806 - dense_2_loss: 0.9708 - dense_3_loss: 1.2906 - dense_4_loss: 0.3931 - dense_5_loss: 0.7554 - dense_6_loss: 1.2708 - dense_2_accuracy: 0.8765 - dense_3_accuracy: 0.8425 - dense_4_accuracy: 0.9637 - dense_5_accuracy: 0.9136 - dense_6_accuracy: 0.8128


[4.680647850036621,
 0.9707695245742798,
 1.2905772924423218,
 0.39306971430778503,
 0.7554128766059875,
 1.270817756652832,
 0.876471996307373,
 0.8424657583236694,
 0.9637106657028198,
 0.9136024713516235,
 0.8127853870391846]

In [52]:
model.save('vuldet.h5')

In [51]:
model = tf.keras.models.load_model("vuldet")

In [7]:

for col in range(1,6):
    print('\nThis is evaluation for column',col)
    confusion = sklearn.metrics.confusion_matrix(y_train=test.iloc[:,col].to_numpy(), y_pred=predicted[col-1])
    print(confusion)
    tn, fp, fn, tp = confusion.ravel()
    print('\nTP:',tp)
    print('FP:',fp)
    print('TN:',tn)
    print('FN:',fn)

    ## Performance measure
#     print('\nAccuracy: '+ str(sklearn.metrics.accuracy_score(y_true=test.iloc[:,col].to_numpy(), y_pred=pred_test[col-1])))
#     print('Precision: '+ str(sklearn.metrics.precision_score(y_true=test.iloc[:,col].to_numpy(), y_pred=pred_test[col-1])))
#     print('Recall: '+ str(sklearn.metrics.recall_score(y_true=test.iloc[:,col].to_numpy(), y_pred=pred_test[col-1])))
#     print('F-measure: '+ str(sklearn.metrics.f1_score(y_true=test.iloc[:,col].to_numpy(), y_pred=pred_test[col-1])))
#     print('Precision-Recall AUC: '+ str(sklearn.metrics.average_precision_score(y_true=test.iloc[:,col].to_numpy(), y_score=predicted[col-1][:,1])))
#     print('AUC: '+ str(sklearn.metrics.roc_auc_score(y_true=test.iloc[:,col].to_numpy(), y_score=predicted[col-1][:,1])))
#     print('MCC: '+ str(sklearn.metrics.matthews_corrcoef(y_true=test.iloc[:,col].to_numpy(), y_pred=pred_test[col-1])))


This is evaluation for column 1


NameError: name 'test' is not defined

In [6]:
from sklearn import metrics